In [1]:
# === Bootstrap cell: shared config, paths, JSON I/O, and GitHub REST/GraphQL helpers ===

import os, json, time, re, random, pathlib
from typing import Dict, Any, Optional, Tuple, List
from datetime import datetime, timezone

import requests
import yaml
from dotenv import load_dotenv

# -------------------------------------------------------------------
# Locate repo root reliably (works no matter where you open the .ipynb)
# -------------------------------------------------------------------
def _find_repo_root(start: pathlib.Path) -> pathlib.Path:
    cur = start.resolve()
    for _ in range(8):  # walk up to 8 levels
        if (cur / "config" / "config.yaml").exists():
            return cur
        cur = cur.parent
    raise FileNotFoundError("Couldn't locate repo root (no config/config.yaml found upward).")

REPO_ROOT = _find_repo_root(pathlib.Path.cwd())
CONFIG_PATH = REPO_ROOT / "config" / "config.yaml"

# -------------------------------------------------------------------
# Load config + token
# -------------------------------------------------------------------
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    CFG: Dict[str, Any] = yaml.safe_load(f) or {}

load_dotenv(REPO_ROOT / ".env")  # local only; do not commit .env
TOKEN_ENV = (CFG.get("github_token_env") or "GITHUB_TOKEN").strip()
GITHUB_TOKEN = os.getenv(TOKEN_ENV, "").strip()
if not GITHUB_TOKEN:
    raise RuntimeError(f"Missing token in environment variable {TOKEN_ENV}. "
                       f"Create .env from .env.example and set {TOKEN_ENV}=...")

# Repo + output root
REPO = CFG["repo"]  # "owner/name"
OWNER, NAME = REPO.split("/", 1)
OUT_ROOT = pathlib.Path(CFG.get("out_root") or NAME)  # "<name>" if null

# Behavior
OVERWRITE = bool(CFG.get("overwrite", True))
VERBOSE = bool(CFG.get("verbose_logs", True))

# Networking knobs
REQ_TIMEOUT = int(CFG.get("request_timeout_sec", 30))
MAX_RETRIES = int(CFG.get("max_retries", 4))
BACKOFF_BASE_MS = int(CFG.get("backoff_base_ms", 400))
BACKOFF_JITTER_MS = int(CFG.get("backoff_jitter_ms", 250))
RESPECT_RL = bool(CFG.get("respect_rate_limits", True))

# -------------------------------------------------------------------
# Tiny logging helpers
# -------------------------------------------------------------------
def log(msg: str) -> None:
    if VERBOSE:
        print(msg, flush=True)

def warn(msg: str) -> None:
    print(f"⚠️  {msg}", flush=True)

print("Using repo root:", REPO_ROOT)

# -------------------------------------------------------------------
# Canonical output layout helpers
# -------------------------------------------------------------------
def ensure_dir(p: pathlib.Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

def repo_root() -> pathlib.Path:
    root = OUT_ROOT
    ensure_dir(root)
    return root

def tags_all_json() -> pathlib.Path:
    d = repo_root() / "tags"
    ensure_dir(d)
    return d / "tags.all.json"

def series_dir(kind: str, series: str) -> pathlib.Path:
    d = repo_root() / kind / series
    ensure_dir(d)
    return d

def pair_stem(base: str, compare: str) -> str:
    return f"{base}...{compare}"

def pair_json(series: str, stem: str, kind: str) -> pathlib.Path:
    # kind ∈ {"compare","commits","pulls","issues"}
    return series_dir(kind, series) / f"{stem}.{kind}.json"

def capsule_json(series: str, stem: str) -> pathlib.Path:
    return series_dir("commits_pr_issue", series) / f"{stem}.tarce_artifacts.json"

# -------------------------------------------------------------------
# JSON I/O
# -------------------------------------------------------------------
def read_json(path: pathlib.Path) -> Optional[Dict[str, Any]]:
    if not path.exists():
        return None
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def write_json(path: pathlib.Path, payload: Dict[str, Any]) -> None:
    ensure_dir(path.parent)
    if path.exists() and not OVERWRITE:
        log(f"Skip (exists): {path}")
        return
    tmp = path.with_suffix(path.suffix + ".tmp")
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    tmp.replace(path)
    log(f"✓ Wrote {path}")

# -------------------------------------------------------------------
# GitHub HTTP session + rate-limit aware helpers
# -------------------------------------------------------------------
SESSION = requests.Session()
SESSION.headers.update({
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": f"notebook-pipeline/{NAME}"
})

def _parse_reset_epoch(headers: Dict[str, Any]) -> Optional[int]:
    try:
        return int(headers.get("X-RateLimit-Reset") or headers.get("x-ratelimit-reset"))
    except Exception:
        return None

def _maybe_sleep_for_reset(resp: requests.Response) -> None:
    if not RESPECT_RL:
        return
    remaining = resp.headers.get("X-RateLimit-Remaining") or resp.headers.get("x-ratelimit-remaining")
    if remaining is not None and str(remaining).isdigit() and int(remaining) <= 0:
        reset_epoch = _parse_reset_epoch(resp.headers)
        if reset_epoch:
            now = int(time.time())
            delta = max(0, reset_epoch - now) + 1
            warn(f"Rate limit reached. Sleeping ~{delta}s until reset …")
            time.sleep(delta)

def _backoff_sleep(i: int) -> None:
    base = BACKOFF_BASE_MS / 1000.0
    jitter = random.uniform(0, BACKOFF_JITTER_MS / 1000.0)
    time.sleep((2 ** i) * base + jitter)

def rest_get_json(url: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    Generic GET with backoff + rate-limit handling.
    """
    last_err = None
    for i in range(MAX_RETRIES + 1):
        try:
            r = SESSION.get(url, params=params, timeout=REQ_TIMEOUT)
            # Honor 429 Retry-After if present
            if r.status_code == 429:
                ra = r.headers.get("Retry-After")
                if ra and ra.isdigit():
                    wait_s = int(ra)
                    warn(f"429 received. Sleeping {wait_s}s per Retry-After …")
                    time.sleep(wait_s)
                    continue
            if r.status_code >= 500:
                last_err = f"{r.status_code} {r.text[:200]}"
                _backoff_sleep(i)
                continue
            if r.status_code >= 400:
                raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
            _maybe_sleep_for_reset(r)
            return r.json()
        except Exception as e:
            last_err = str(e)
            _backoff_sleep(i)
    raise RuntimeError(f"GET failed after retries: {url} :: {last_err}")

def gh_graphql(query: str, variables: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    GitHub GraphQL POST with backoff + rate-limit handling.
    Endpoint: https://api.github.com/graphql
    """
    url = "https://api.github.com/graphql"
    payload = {"query": query, "variables": variables or {}}
    last_err = None
    for i in range(MAX_RETRIES + 1):
        try:
            r = SESSION.post(url, json=payload, timeout=REQ_TIMEOUT)
            if r.status_code == 429:
                ra = r.headers.get("Retry-After")
                if ra and ra.isdigit():
                    wait_s = int(ra)
                    warn(f"429 received. Sleeping {wait_s}s per Retry-After …")
                    time.sleep(wait_s)
                    continue
            if r.status_code >= 500:
                last_err = f"{r.status_code} {r.text[:200]}"
                _backoff_sleep(i)
                continue
            if r.status_code >= 400:
                raise RuntimeError(f"GraphQL HTTP {r.status_code}: {r.text[:500]}")
            _maybe_sleep_for_reset(r)
            data = r.json()
            if "errors" in data:
                last_err = f"GraphQL errors: {data['errors']}"
                _backoff_sleep(i)
                continue
            return data
        except Exception as e:
            last_err = str(e)
            _backoff_sleep(i)
    raise RuntimeError(f"GraphQL failed after retries: {last_err}")

def rate_limit_snapshot() -> Dict[str, Any]:
    """
    Returns current REST rate-limit bucket (printed; not written to JSON outputs).
    """
    try:
        info = rest_get_json("https://api.github.com/rate_limit")
        core = info.get("resources", {}).get("core", {})
        remaining = core.get("remaining")
        limit = core.get("limit")
        reset = core.get("reset")
        when = datetime.fromtimestamp(reset, tz=timezone.utc).isoformat() if reset else None
        snap = {"limit": limit, "remaining": remaining, "reset_epoch": reset, "reset_iso": when}
        log(f"Rate limit: {remaining}/{limit}, resets at {when}")
        return snap
    except Exception as e:
        warn(f"Rate limit snapshot failed: {e}")
        return {}

# -------------------------------------------------------------------
# Light helpers for series + ordering
# -------------------------------------------------------------------
def semver_series(tag_name: str) -> str:
    """
    Extract 'vX.Y' series from tags like 'v4.2.3', '4.2.0-rc.1', etc.
    Falls back to 'v0.0' if not parseable.
    """
    m = re.search(r'v?(\d+)\.(\d+)', tag_name or "")
    if not m:
        return "v0.0"
    return f"v{int(m.group(1))}.{int(m.group(2))}"

def sorted_pairs_by_tag_time(pairs: List[Tuple[str, str]], tag_index: Dict[str, Dict[str, Any]]) -> List[Tuple[str, str]]:
    """
    Sort (base, compare) by tag timestamps ascending when both are tags;
    unknown timestamps sort last.
    """
    def ts(tag: str) -> float:
        rec = tag_index.get(tag)
        if rec and rec.get("tag_timestamp"):
            try:
                return datetime.fromisoformat(rec["tag_timestamp"].replace("Z", "+00:00")).timestamp()
            except Exception:
                pass
        return float("inf")
    return sorted(pairs, key=lambda bc: (ts(bc[0]), ts(bc[1])))

log("Bootstrap ready ✓")


Using repo root: /Users/rambodparsi/Desktop/OSS Repository Selection/web_scrapper/commit_pr_issue_analysis
Bootstrap ready ✓


In [2]:
# M7 — Build per-major-version JSON for LLM input from final capsules
from pathlib import Path
from collections import OrderedDict
import json

# 1) Discover capsules
caps_base = Path(repo_root()) / "commits_pr_issue"
if not caps_base.exists():
    raise FileNotFoundError(f"Not found: {caps_base}. Run M6 first.")

capsules = sorted(caps_base.rglob("*.tarce_artifacts.json"))
print("Capsule files found:", len(capsules))

# Optional: whitelist series from config
SERIES_WHITELIST = CFG.get("series_whitelist") or []
if SERIES_WHITELIST:
    before = len(capsules)
    capsules = [p for p in capsules if p.parent.name in set(SERIES_WHITELIST)]
    print(f"Applied series_whitelist={SERIES_WHITELIST} → {len(capsules)}/{before} remain")

# 2) Group capsules by series
by_series = {}
for p in capsules:
    series = p.parent.name  # e.g., v4.2
    by_series.setdefault(series, []).append(p)

print("Series present:", sorted(by_series.keys()))

# 3) For each series, collate commits → single PR/issue fields
out_dir = Path(repo_root()) / "gemini_input"
out_dir.mkdir(parents=True, exist_ok=True)

written = 0
for series, files in sorted(by_series.items()):
    # sha -> record (first write wins)
    sha_map = OrderedDict()

    for f in sorted(files):
        data = read_json(f) or {}
        capsules = data.get("commit_pr_issue") or []

        for cap in capsules:
            commit = (cap.get("commit") or [{}])[0]
            sha = commit.get("sha")
            if not sha or sha in sha_map:
                continue

            # Commit title
            commit_title = commit.get("commit_title") or ""

            # Primary PR (first)
            prs = cap.get("prs") or []
            if prs:
                pr0 = prs[0]
                pr_title = pr0.get("pr_title") or ""
                pr_body  = pr0.get("pr_body") or ""
                pr_labels = list(pr0.get("labels") or [])
            else:
                pr_title = ""
                pr_body = ""
                pr_labels = []

            # Primary Issue (first; M6 already ordered closes-first)
            issues = cap.get("issues") or []
            if issues:
                is0 = issues[0]
                issue_title = is0.get("issue_title") or ""
                issue_body  = is0.get("issue_body") or ""
                issue_labels = list(is0.get("labels") or [])
            else:
                issue_title = ""
                issue_body = ""
                issue_labels = []

            sha_map[sha] = {
                "commit_title": commit_title,
                "pr_title": pr_title,
                "pr_body": pr_body,
                "pr_label": pr_labels,
                "issue_title": issue_title,
                "issue_body": issue_body,
                "issue_label": issue_labels
            }

    # 4) Write one file per major series
    out_path = out_dir / f"{series}.json"
    if out_path.exists() and not OVERWRITE:
        print(f"Skip (exists): {out_path}")
        continue

    with open(out_path, "w", encoding="utf-8") as fh:
        json.dump(sha_map, fh, indent=2, ensure_ascii=False)

    print("Wrote:", out_path)
    written += 1

print(f"Done. Series written: {written}")


Capsule files found: 107
Applied series_whitelist=['v4.0'] → 19/107 remain
Series present: ['v4.0']
Wrote: mastodon/gemini_input/v4.0.json
Done. Series written: 1


In [3]:
# M7 (extra) — Write CSVs from gemini_input JSONs
from pathlib import Path
import json, csv

gi_dir = Path(repo_root()) / "gemini_input"
gi_dir.mkdir(parents=True, exist_ok=True)

series_jsons = sorted(gi_dir.glob("v*.json"))
print("Found series JSONs:", [p.name for p in series_jsons])

# helper to write one CSV
def write_series_csv(json_path: Path) -> Path:
    series = json_path.stem  # e.g., v4.2
    out_csv = gi_dir / f"{series}.csv"
    with open(json_path, "r", encoding="utf-8") as fh:
        data = json.load(fh)  # { sha: {...} }

    with open(out_csv, "w", newline="", encoding="utf-8") as fh:
        w = csv.writer(fh)
        w.writerow(["sha", "issue_title", "issue_body", "issue_lable"])
        for sha, row in data.items():
            ititle = row.get("issue_title") or ""
            ibody  = row.get("issue_body") or ""
            ilabs  = row.get("issue_label") or []  # stored as list in M7
            # join labels into a single cell
            ilabs_str = ", ".join(ilabs) if isinstance(ilabs, list) else (ilabs or "")
            w.writerow([sha, ititle, ibody, ilabs_str])
    print("Wrote:", out_csv)
    return out_csv

# per-series CSVs
csv_paths = [write_series_csv(p) for p in series_jsons]

# optional: combined CSV across all series
all_csv = gi_dir / "all_series.csv"
with open(all_csv, "w", newline="", encoding="utf-8") as fh_all:
    w_all = csv.writer(fh_all)
    w_all.writerow(["sha", "issue_title", "issue_body", "issue_lable"])
    for p in series_jsons:
        with open(p, "r", encoding="utf-8") as fh:
            data = json.load(fh)
        for sha, row in data.items():
            ititle = row.get("issue_title") or ""
            ibody  = row.get("issue_body") or ""
            ilabs  = row.get("issue_label") or []
            ilabs_str = ", ".join(ilabs) if isinstance(ilabs, list) else (ilabs or "")
            w_all.writerow([sha, ititle, ibody, ilabs_str])
print("Wrote:", all_csv)


Found series JSONs: ['v4.0.json', 'v4.1.json', 'v4.2.json', 'v4.3.json', 'v4.4.json']
Wrote: mastodon/gemini_input/v4.0.csv
Wrote: mastodon/gemini_input/v4.1.csv
Wrote: mastodon/gemini_input/v4.2.csv
Wrote: mastodon/gemini_input/v4.3.csv
Wrote: mastodon/gemini_input/v4.4.csv
Wrote: mastodon/gemini_input/all_series.csv
